# 不通过`wallet`发送交易

Run this example online by clicking `🚀` -&gt; `Binder` on the top bar!

This example shows how to sign and send transactions without using `w3.wallet`. This method is more flexible but simultaneously more complex to use. If you are new to web3 development, we recommend you refer to the example to send transactions in [quickstart](./01-quickstart.ipynb)

In [1]:
import pprint
from conflux_web3 import Web3
from conflux_web3.utils import fill_transaction_defaults

w3 = Web3(Web3.HTTPProvider("https://test.confluxrpc.com"))
account = w3.account.from_key("0x....") # fill your secret key here

## Manully Send a Transaction

Basiclly, we follow the workflow of 4 steps to send a transaction:

* build transaction
* sign transaction
* send transaction to the blockchain node(s)
* wait for transaction execution result

If any account is added to the `w3.wallet`, the first 3 steps will be combined,
otherwise you will need to do the first 3 steps manully

### 构造一个简单的交易

Only necessary parameters are provided here, and other params are filled by `fill_transaction_defaults`. Refer to [constuct_transaction_from_scratch](./11-constuct_transaction_from_scratch.ipynb) to see how to construct a transaction from scratch.

In [2]:
built_trivial_tx = fill_transaction_defaults(w3, {
    'from': account.address,
    'to': w3.account.create().address,
    'value': 100,
})

### 为交易签名

In [3]:
signed_tx = account.sign_transaction(built_trivial_tx)

### Send the Transaction

In [4]:
h = w3.cfx.send_raw_transaction(signed_tx.rawTransaction)

### Wait

In [5]:
tx_receipt = h.executed()

## Interact with Contract without Wallet

In [6]:
# if you want to get contract object from metadata file, use
# >>> erc20_metadata = json.load(open("path/to/ERC20metadata.json"))
# >>> erc20 = web3.cfx.contract(bytecode=erc20_metadata["bytecode"], abi=erc20_metadata["abi"])
erc20 = w3.cfx.contract(name="ERC20")

# "build transaction"
built_constuct_tx = erc20.constructor(name="Coin", symbol="C", initialSupply=10**18).build_transaction({
    'from': account.address,
})
# "sign transaction"
construct_tx = account.sign_transaction(built_constuct_tx)
# "send" & "wait"
contract_address = w3.cfx.send_raw_transaction(construct_tx.rawTransaction).executed()['contractCreated']
print(f"deployed contract address: {contract_address}")

deployed contract address: cfxtest:acfgtrw53v4ju9jaydmr9yyf9xyvfxpe0e8ny2fd4d


In [7]:
# interact with the deployed contract
contract = w3.cfx.contract(address=contract_address, name="ERC20")
# "build transaction"
built_transfer_tx = contract.functions.transfer(
    w3.account.create().address,
    100
).build_transaction({
    'from': account.address
})
# "sign transaction"
signed_transfer_tx = account.sign_transaction(
    built_transfer_tx
)
# "send" and "wait"
print("erc20 transfer receipt: ")
pprint.pprint(
    dict(w3.cfx.send_raw_transaction(signed_transfer_tx.rawTransaction).executed())
)

erc20 transfer receipt: 
{'blockHash': HexBytes('0x9b8a0a0e5eb33d67b2c85d4e951b0251e1b972cd00f7c18c53ecacb7fd7edfd3'),
 'contractCreated': None,
 'epochNumber': 99051304,
 'from': 'cfxtest:aanhtnrex2nj56kkbws4yx0jeab34ae16pcap53w13',
 'gasCoveredBySponsor': False,
 'gasFee': 37063000000000 Drip,
 'gasUsed': 37063,
 'index': 0,
 'logs': [AttributeDict({'address': 'cfxtest:acfgtrw53v4ju9jaydmr9yyf9xyvfxpe0e8ny2fd4d', 'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'), HexBytes('0x0000000000000000000000001677ada49e168df1290c9daa4ec820039d0097e3'), HexBytes('0x0000000000000000000000001ab7c2bb12a3484451a55ed15285a4cf10196312')], 'data': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000064')})],
 'logsBloom': HexBytes('0x00000020000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000008000000000000000000000000000000000000200008000000000000000000000000000000000000000000000000000000000000